In [ ]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
import joblib
from flask import Flask, request, jsonify

# Step 2: Load and Preprocess Data
df = pd.read_csv('network_traffic.csv')  # Replace with your dataset
df.dropna(inplace=True)
df = pd.get_dummies(df)  # Encode categorical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df)
X_train, X_test = train_test_split(scaled_features, test_size=0.2, random_state=42)

# Step 3: Train Isolation Forest Model
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Step 4: Make Predictions
y_pred = iso_forest.predict(X_test)
y_pred_binary = [1 if x == -1 else 0 for x in y_pred]

# Optional: True labels (only if available, else skip classification report)
# y_true = ...
# print(classification_report(y_true, y_pred_binary))

# Step 5: Visualize Anomalies
viz_df = pd.DataFrame(X_test, columns=df.columns)
viz_df['Anomaly'] = y_pred_binary
sns.pairplot(viz_df.sample(200), hue='Anomaly', palette='Set1')
plt.show()

# Step 6: Save Model and Scaler
joblib.dump(iso_forest, 'isolation_forest_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Step 7: Create Flask API
app = Flask(_name_)
model = joblib.load('isolation_forest_model.pkl')
scaler = joblib.load('scaler.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = np.array(data['features']).reshape(1, -1)
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)
    anomaly = 1 if prediction[0] == -1 else 0
    return jsonify({'anomaly': anomaly})

if _name_ == '_main_':
    app.run(debug=True)